Run experiments to compare quadratic underestimators with other type of estimators.


In [51]:
import pandas as pd
import numpy as np
import interval as ival
import uvarprob as uvpr
import bnb as bnb
import sub as sub
import time
import ivalprocessor as ivproc
import pslprocessor as pslproc
import psqeprocessor as psqproc
import gridsearch as gs
from sortedcontainers import SortedKeyList
from collections import namedtuple
from pathlib import Path

Define an array of test files to process

In [26]:
test_files = ["/tmp/shek.csv"]

Define some tuning parameters

In [27]:
# Maximal number of steps
max_steps = 1e6
# Tolerance
epsilon = 1e-2
# If True - global Lipschitz constant is used
global_lipschitz_interval = True
# If True - the record value is taken from the test database
known_record = False

In [28]:
def get_initial_recval(prob):
    if known_record:
        return prob.min_f
    else:
        return float('inf')

In [29]:
TestResult = namedtuple('TestResult', ['nsteps', 'record_value'])

In [30]:
def psl(prob, sym):
    psp = pslproc.PSLProcessor(rec_v = get_initial_recval(prob), rec_x = None, problem = prob, eps = epsilon, global_lipint = global_lipschitz_interval, use_symm_lipint = sym)
    sl = SortedKeyList(key = lambda s : s.level)
    subp = sub.Sub(0, 0, pslproc.PSLData(ival.Interval([prob.a, prob.b]),0))
    psp.compute_bounds(subp)
    sl.add(subp)
    cnt = max_steps
    steps = bnb.bnb(sl, cnt, psp)
#     print("steps performed: ", steps)
#     print("Record value = ", psp.rec_v, " at ", psp.rec_x);
#     print(sl)
    return TestResult(nsteps = steps, record_value = psp.rec_v)

In [31]:
def psqe(prob, sym):
    psp = psqproc.PSQEProcessor(rec_v = get_initial_recval(prob), rec_x = None, problem = prob, eps = epsilon, global_lipint = global_lipschitz_interval, use_symm_lipint = sym)
    sl = SortedKeyList(key = lambda s : s.level)
    subp = sub.Sub(0, 0, psqproc.PSQEData(ival.Interval([prob.a, prob.b]),0))
    psp.compute_bounds(subp)
    sl.add(subp)
    cnt = max_steps
    steps = bnb.bnb(sl, cnt, psp)
#     print("steps performed: ", steps)
#     print("Record value = ", psp.rec_v, " at ", psp.rec_x);
#     print(sl)
    return TestResult(nsteps = steps, record_value = psp.rec_v)

In [32]:
def psl_non_sym(prob):
    return psl(prob, False)

In [33]:
def psl_sym(prob):
    return psl(prob, True)

In [34]:
def psqe_non_sym(prob):
    return psqe(prob, False)

In [35]:
def psqe_sym(prob):
    return psqe(prob, True)

Tested methods

In [36]:
test_methods = {'LLI' : psl_non_sym, 'LLC' : psl_sym, 'QLI' : psqe_non_sym, 'QLC' : psqe_sym}

In [37]:
# test_methods = {'LLI' : psl_non_sym, 'QLI' : psqe_non_sym, 'QLC' : psqe_sym}

In [38]:
def run_method(method, prob):
    result = method(prob)
    return result

In [39]:
def run_test(test):
    prob = uvpr.UniVarProblem(test.Index, test.objective, test.a, test.b, test.min_f, test.min_x)
    print(prob.min_f)
    runs = {}
    for name, method in test_methods.items():
        result = run_method(method, prob)
#         print(name, "result :", result)
        runs[name] = result
    return runs



In [40]:
def run_tests(df):
    data = {}
    for row in df.itertuples():
        for name, tres in run_test(row).items():
            steps_key = name + '_steps'
            if data.get(steps_key) == None:
                data[steps_key] = [tres.nsteps]
            else:
                data[steps_key].append(tres.nsteps)
            record_key = name + '_record'
            if data.get(record_key) == None:
                data[record_key] = [tres.record_value]
            else:
                data[record_key].append(tres.record_value)
#     print(data)
    perf_df = pd.DataFrame(data, index = df.index)
#     print(perf_df)
#     print(perf_df.diff(axis = 1, periods = 2))
    return perf_df

In [41]:
def read_problems(fname):
    data = pd.read_csv(fname, index_col = 'name', comment = '#')
    return data

Main testing scenario

In [53]:
for file in test_files:
    start = time.time()
    df = run_tests(read_problems(file))
    end = time.time()
    print("elapsed:", end - start)
    filename = Path(file)
    filename_wo_ext = filename.with_suffix('')
    df.to_csv(str(filename_wo_ext) + "_res.csv")
    
    

0.53993
1.4095
elapsed: 90.51236963272095


In [54]:
csv_name = "/tmp/shek.csv_results"
df = pd.read_csv(csv_name)
print(df)
filtered_df = df[['LLC_steps', 'LLI_steps', 'QLC_steps', 'QLI_steps']]
print(filtered_df)
ratio = filtered_df['QLC_steps']/filtered_df['QLI_steps']
print("ratio")
print(type(ratio))
print(ratio)
print("ratio:", ratio.min(), ratio.mean(), ratio.max())
mean_data = filtered_df.mean()
max_data = filtered_df.max()
min_data = filtered_df.min()
#     print(mean_data['LLI_steps'])
print("=============")
#     steps_data = pd.Series(mean_data, index = ['LLC_steps', 'LLI_steps', 'QLC_steps', 'QLI_steps'])
print("Mean:")
print(mean_data)
print("Min:")
print(min_data)
print("Max:")
print(max_data)

        name  LLI_steps  LLI_record  LLC_steps  LLC_record  QLI_steps  \
0  rshekel_0     551839    0.539925     754797    0.539925      18426   
1  rshekel_1     309368    1.409502     295817    1.409502      10822   

   QLI_record  QLC_steps  QLC_record  
0    0.539925      23631    0.539925  
1    1.409502      16093    1.409502  
   LLC_steps  LLI_steps  QLC_steps  QLI_steps
0     754797     551839      23631      18426
1     295817     309368      16093      10822
ratio
<class 'pandas.core.series.Series'>
0    1.282481
1    1.487063
dtype: float64
ratio: 1.28248127645718 1.3847723329245798 1.4870633893919794
Mean:
LLC_steps    525307.0
LLI_steps    430603.5
QLC_steps     19862.0
QLI_steps     14624.0
dtype: float64
Min:
LLC_steps    295817
LLI_steps    309368
QLC_steps     16093
QLI_steps     10822
dtype: int64
Max:
LLC_steps    754797
LLI_steps    551839
QLC_steps     23631
QLI_steps     18426
dtype: int64
